<a href="https://colab.research.google.com/github/RosaR02/Sales-Predictions/blob/main/SalesPredictionV.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [72]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.compose import make_column_selector, make_column_transformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder, OrdinalEncoder
from sklearn.pipeline import make_pipeline
from sklearn.impute import SimpleImputer

from sklearn import set_config
set_config(display='diagram')

In [73]:
# or your path here
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/sales_predictions.csv')
df.head()

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales
0,FDA15,9.30,Low Fat,0.016047,Dairy,249.8092,OUT049,1999,Medium,Tier 1,Supermarket Type1,3735.1380
1,DRC01,5.92,Regular,0.019278,Soft Drinks,48.2692,OUT018,2009,Medium,Tier 3,Supermarket Type2,443.4228
2,FDN15,17.50,Low Fat,0.016760,Meat,141.6180,OUT049,1999,Medium,Tier 1,Supermarket Type1,2097.2700
3,FDX07,19.20,Regular,0.000000,Fruits and Vegetables,182.0950,OUT010,1998,NaN,Tier 3,Grocery Store,732.3800
4,NCD19,8.93,Low Fat,0.000000,Household,53.8614,OUT013,1987,High,Tier 3,Supermarket Type1,994.7052


In [74]:
df.duplicated().sum()

0

In [75]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8523 entries, 0 to 8522
Data columns (total 12 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Item_Identifier            8523 non-null   object 
 1   Item_Weight                7060 non-null   float64
 2   Item_Fat_Content           8523 non-null   object 
 3   Item_Visibility            8523 non-null   float64
 4   Item_Type                  8523 non-null   object 
 5   Item_MRP                   8523 non-null   float64
 6   Outlet_Identifier          8523 non-null   object 
 7   Outlet_Establishment_Year  8523 non-null   int64  
 8   Outlet_Size                6113 non-null   object 
 9   Outlet_Location_Type       8523 non-null   object 
 10  Outlet_Type                8523 non-null   object 
 11  Item_Outlet_Sales          8523 non-null   float64
dtypes: float64(4), int64(1), object(7)
memory usage: 799.2+ KB


In [76]:
df.isna().sum().sum()

3873

In [77]:
df.isna().sum()

Item_Identifier                 0
Item_Weight                  1463
Item_Fat_Content                0
Item_Visibility                 0
Item_Type                       0
Item_MRP                        0
Outlet_Identifier               0
Outlet_Establishment_Year       0
Outlet_Size                  2410
Outlet_Location_Type            0
Outlet_Type                     0
Item_Outlet_Sales               0
dtype: int64

In [78]:
df.describe(include='number')

,Item_Weight,Item_Visibility,Item_MRP,Outlet_Establishment_Year,Item_Outlet_Sales
count,7060.000000,8523.000000,8523.000000,8523.000000,8523.000000
mean,12.857645,0.066132,140.992782,1997.831867,2181.288914
std,4.643456,0.051598,62.275067,8.371760,1706.499616
min,4.555000,0.000000,31.290000,1985.000000,33.290000
25%,8.773750,0.026989,93.826500,1987.000000,834.247400
50%,12.600000,0.053931,143.012800,1999.000000,1794.331000
75%,16.850000,0.094585,185.643700,2004.000000,3101.296400
max,21.350000,0.328391,266.888400,2009.000000,13086.964800


In [79]:
df.describe(include='object')

,Item_Identifier,Item_Fat_Content,Item_Type,Outlet_Identifier,Outlet_Size,Outlet_Location_Type,Outlet_Type
count,8523,8523,8523,8523,6113,8523,8523
unique,1559,5,16,10,3,3,4
top,FDW13,Low Fat,Fruits and Vegetables,OUT027,Medium,Tier 3,Supermarket Type1
freq,10,5089,1232,935,2793,3350,5577


In [80]:
df.drop(columns='Item_Identifier')

,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales
0,9.300,Low Fat,0.016047,Dairy,249.8092,OUT049,1999,Medium,Tier 1,Supermarket Type1,3735.1380
1,5.920,Regular,0.019278,Soft Drinks,48.2692,OUT018,2009,Medium,Tier 3,Supermarket Type2,443.4228
2,17.500,Low Fat,0.016760,Meat,141.6180,OUT049,1999,Medium,Tier 1,Supermarket Type1,2097.2700
3,19.200,Regular,0.000000,Fruits and Vegetables,182.0950,OUT010,1998,NaN,Tier 3,Grocery Store,732.3800
4,8.930,Low Fat,0.000000,Household,53.8614,OUT013,1987,High,Tier 3,Supermarket Type1,994.7052
...,...,...,...,...,...,...,...,...,...,...,...
8518,6.865,Low Fat,0.056783,Snack Foods,214.5218,OUT013,1987,High,Tier 3,Supermarket Type1,2778.3834
8519,8.380,Regular,0.046982,Baking Goods,108.1570,OUT045,2002,NaN,Tier 2,Supermarket Type1,549.2850
8520,10.600,Low Fat,0.035186,Health and Hygiene,85.1224,OUT035,2004,Small,Tier 2,Supermarket Type1,1193.1136
8521,7.210,Regular,0.145221,Snack Foods,103.1332,OUT018,2009,Medium,Tier 3,Supermarket Type2,1845.5976


In [81]:
df['Item_Outlet_Sales'].isna().sum()

0

In [82]:
df['Outlet_Size'].unique()

array(['Medium', nan, 'High', 'Small'], dtype=object)

In [83]:
#df['Item_Weight'].unique()

In [84]:
# Ordered Category Lists
Outlet_Size_list = ['Small', 'Medium', 'High']

# Transformers
ord = OrdinalEncoder(categories=[Outlet_Size_list])
freq_imputer = SimpleImputer(strategy='most_frequent')
scaler2 = StandardScaler()
# Pipeline
ord_pipe = make_pipeline(freq_imputer, ord, scaler2)
# Selector
ord_cols = ['Outlet_Size']
# Tuple
ord_tuple = (ord_pipe, ord_cols)

In [85]:
# replacing inconsistent categories in Outlet Size List
# then checking to make sure the change took place

#Outlet_Size = {'Small':'0',
 #                  'Medium':'1',
 #                  'High':'2'}

#df['Outlet_Size'] = df['Outlet_Size'].replace(Outlet_Size)
#df['Outlet_Size'].value_counts()

In [86]:
# split X and y, we are predicting future sales
X = df.drop(columns=['Item_Identifier', 'Outlet_Identifier', 'Item_Outlet_Sales'])
y = df['Item_Outlet_Sales']
# split training and test
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [87]:
X_train.shape

(6392, 9)

In [88]:
X_test.shape

(2131, 9)

In [89]:
#print(Outlet_Size_list)

In [90]:
# Transformers
mean_imputer = SimpleImputer(strategy='mean')
scaler = StandardScaler()

# Pipeline
num_pipe = make_pipeline(mean_imputer, scaler)

# Selector
num_selector = make_column_selector(dtype_include='number')

# Tuple
num_tuple = (num_pipe, num_selector)

In [91]:
# Instantiate the make column transformer
preprocessor = make_column_transformer(ord_tuple,
                                       num_tuple,
                                       remainder='drop',
                                       verbose_feature_names_out=False)

In [92]:
#print(X)

In [93]:
#print(X_train)

In [94]:
preprocessor.fit(X_train)

ColumnTransformer(transformers=[('pipeline-1',
                                 Pipeline(steps=[('simpleimputer',
                                                  SimpleImputer(strategy='most_frequent')),
                                                 ('ordinalencoder',
                                                  OrdinalEncoder(categories=[['Small',
                                                                              'Medium',
                                                                              'High']])),
                                                 ('standardscaler',
                                                  StandardScaler())]),
                                 ['Outlet_Size']),
                                ('pipeline-2',
                                 Pipeline(steps=[('simpleimputer',
                                                  SimpleImputer()),
                                                 ('standardscaler',
                                                  StandardScaler())]),
                                 <sklearn.compose._column_transformer.make_column_selector object at 0x7fe43e3f7490>)],
                  verbose_feature_names_out=False)

In [95]:
X_train_proc = preprocessor.transform(X_train)
X_test_proc = preprocessor.transform(X_test)

In [96]:
X_train_proc

array([[ 0.28737371,  0.81724868, -0.71277507,  1.82810922,  1.32784893],
       [ 0.28737371,  0.5563395 , -1.29105225,  0.60336888,  1.32784893],
       [ 0.28737371, -0.13151196,  1.81331864,  0.24454056,  0.13618724],
       ...,
       [ 0.28737371,  1.11373638, -0.92052713,  1.52302674,  0.49368575],
       [ 0.28737371,  1.76600931, -0.2277552 , -0.38377708,  1.0895166 ],
       [-1.38404828,  0.81724868, -0.95867683, -0.73836105, -0.10214509]])

In [97]:
# Create dataframes from the processed arrays
# First get column names
cols = preprocessor.get_feature_names_out()
# Then transform numpy arrays to dataframes
X_train_df = pd.DataFrame(X_train_proc, columns=cols)
X_train_df

,Outlet_Size,Item_Weight,Item_Visibility,Item_MRP,Outlet_Establishment_Year
0,0.287374,0.817249,-0.712775,1.828109,1.327849
1,0.287374,0.556340,-1.291052,0.603369,1.327849
2,0.287374,-0.131512,1.813319,0.244541,0.136187
3,-1.384048,-1.169219,-1.004931,-0.952591,0.732018
4,0.287374,1.528819,-0.965484,-0.336460,0.493686
...,...,...,...,...,...
6387,0.287374,-0.832409,4.309657,-0.044657,0.017021
6388,0.287374,0.639356,1.008625,-1.058907,1.089517
6389,0.287374,1.113736,-0.920527,1.523027,0.493686
6390,0.287374,1.766009,-0.227755,-0.383777,1.089517


In [98]:
X_train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6392 entries, 0 to 6391
Data columns (total 5 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Outlet_Size                6392 non-null   float64
 1   Item_Weight                6392 non-null   float64
 2   Item_Visibility            6392 non-null   float64
 3   Item_MRP                   6392 non-null   float64
 4   Outlet_Establishment_Year  6392 non-null   float64
dtypes: float64(5)
memory usage: 249.8 KB
